# Collision Avoidance - Live Demo

In this notebook we'll use the model we trained to detect whether the robot is ``free`` or ``blocked`` to enable a collision avoidance behavior on the robot.  

## Load the trained model

We'll assumed that you've already downloaded the ``best_model.pth`` to your workstation as instructed in the training notebook.  Now, you should upload this model into this notebook's
directory by using the Jupyter Lab upload tool.  Once that's finished there should be a file named ``best_model.pth`` in this notebook's directory.  

> Please make sure the file has uploaded fully before calling the next cell

Execute the code below to initialize the PyTorch model.  This should look very familiar from the training notebook.

## Importing the object detection model


In [1]:
from models import *
from utils import *

import os, sys, time, datetime, random
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.autograd import Variable

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import cv2
import time

config_path='config/yolov3.cfg'
weights_path='config/yolov3.weights'
class_path='config/coco.names'

img_size=416
conf_thres=0.8
nms_thres=0.4

# Load model and weights
model = Darknet(config_path, img_size=img_size)
model.load_weights(weights_path)
model.cuda()
model.eval()
classes = utils.load_classes(class_path)
Tensor = torch.cuda.FloatTensor


def detect_image(img):
    # scale and pad image
    ratio = min(img_size/img.size[0], img_size/img.size[1])
    imw = round(img.size[0] * ratio)
    imh = round(img.size[1] * ratio)
    print("transforming image")
    img_transforms = transforms.Compose([ transforms.Resize((imh, imw)),
         transforms.Pad((max(int((imh-imw)/2),0), max(int((imw-imh)/2),0), max(int((imh-imw)/2),0), max(int((imw-imh)/2),0)),
                        (128,128,128)),
         transforms.ToTensor(),
         ])
    print("Transformation done")
    
    # convert image to Tensor
    image_tensor = img_transforms(img).float()
    image_tensor = image_tensor.unsqueeze_(0)
    input_img = Variable(image_tensor.type(Tensor))
    
    # run inference on the model and get detections
    print("Starting inference")
    with torch.no_grad():
        detections = model(input_img)
        detections = utils.non_max_suppression(detections, 80, conf_thres, nms_thres)
    print("inference finished")

    return detections[0]


# load image and get detections
def find_object(object_name,img,selection='object_only',visualize=False, style='line'):
    prev_time = time.time()
    object_found=False
    #Detecting objects
    detections = detect_image(img)

    inference_time = datetime.timedelta(seconds=time.time() - prev_time)
    
    object_dict={}
    
    if detections is not None:
        start_time = time.time()
        unique_labels = detections[:, -1].cpu().unique()
        n_cls_preds = len(unique_labels)

        #converting the image to numpy array
        img = np.array(img)
        
        pad_x = max(img.shape[0] - img.shape[1], 0) * (img_size / max(img.shape))
        pad_y = max(img.shape[1] - img.shape[0], 0) * (img_size / max(img.shape))
        unpad_h = img_size - pad_y
        unpad_w = img_size - pad_x
        
        if visualize==True:
            cmap = plt.get_cmap('tab20b')
            colors = [cmap(i) for i in np.linspace(0, 1, 20)]
            bbox_colors = random.sample(colors, n_cls_preds)
            plt.figure()
            fig, ax = plt.subplots(1, figsize=(12,9))
            ax.imshow(img)

        for x1, y1, x2, y2, conf, cls_conf, cls_pred in detections:
            if selection=='object_only' and classes[int(cls_pred)]!=object_name:
                continue
            else:
                object_found=True
                
            box_h = ((y2 - y1) / unpad_h) * img.shape[0]
            box_w = ((x2 - x1) / unpad_w) * img.shape[1]
            y1 = ((y1 - pad_y // 2) / unpad_h) * img.shape[0]
            x1 = ((x1 - pad_x // 2) / unpad_w) * img.shape[1]

            if visualize==True:
                color = bbox_colors[int(np.where(unique_labels == int(cls_pred))[0])]
                if style=='box':
                    bbox = patches.Rectangle((x1, y1), box_w, box_h, linewidth=2, edgecolor=color, facecolor='none')
                else:
                    bbox = patches.Rectangle((x1, y1+box_h), box_w, 0, linewidth=2, edgecolor=color, facecolor='none')
                ax.add_patch(bbox)
                plt.text(x1, y1, s=classes[int(cls_pred)], color='white', verticalalignment='top',bbox={'color': color, 'pad': 0})

            object_dict[classes[int(cls_pred)]]=(classes[int(cls_pred)],((x1+box_h).item(),(y1).item()),((x1+box_h+box_w).item(),(y1).item()))
            if object_found:
                break
            
        if visualize==True:
            plt.axis('off')
            plt.show()
    return object_dict


/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:47: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


In [2]:
import torch
import torchvision

model_oa = torchvision.models.alexnet(pretrained=False)
model_oa.classifier[6] = torch.nn.Linear(model_oa.classifier[6].in_features, 2)

Next, load the trained weights from the ``best_model.pth`` file that you uploaded

In [3]:
model_oa.load_state_dict(torch.load('best_model_owen.pth'))

Currently, the model weights are located on the CPU memory execute the code below to transfer to the GPU device.

In [4]:
device = torch.device('cuda')
model_oa = model_oa.to(device)

### Create the preprocessing function

We have now loaded our model, but there's a slight issue.  The format that we trained our model doesnt *exactly* match the format of the camera.  To do that, 
we need to do some *preprocessing*.  This involves the following steps

1. Convert from BGR to RGB
2. Convert from HWC layout to CHW layout
3. Normalize using same parameters as we did during training (our camera provides values in [0, 255] range and training loaded images in [0, 1] range so we need to scale by 255.0
4. Transfer the data from CPU memory to GPU memory
5. Add a batch dimension

In [5]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

Great! We've now defined our pre-processing function which can convert images from the camera format to the neural network input format.

Now, let's start and display our camera.  You should be pretty familiar with this by now.  We'll also create a slider that will display the
probability that the robot is blocked.

In [6]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
#from jetbot import Camera, bgr8_to_jpeg


from jetcam.csi_camera import CSICamera
from jetcam.utils import bgr8_to_jpeg
camera = CSICamera(width=225, height=225, fps=2)
camera.running = True


image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, blocked_slider]))

We'll also create our robot instance which we'll need to drive the motors.

In [7]:
from jetracer.nvidia_racecar import NvidiaRacecar
car = NvidiaRacecar()
car.steering=0


In [ ]:
#Testing object detection
from PIL import Image
im = Image.fromarray(camera.value)
im.save("test_img.jpg")

img_path = "test_img.jpg"
img = Image.open(img_path)
detections=find_object('cup',img,selection='object_only',visualize=True, style='line')
print(detections)

Next, we'll create a function that will get called whenever the camera's value changes.  This function will do the following steps

1. Pre-process the camera image
2. Execute the neural network
3. While the neural network output indicates we're blocked, we'll turn left, otherwise we go forward.

In [8]:
import torch.nn.functional as F
import time

def turn(degrees,direction='left'):
    if direction!='left':
        direction=-1
    else:
        direction=1
        
    turn_first=-1
    turns_needed=int(degrees/20)
    
    for i in range(turns_needed):
        car.steering=1*direction
        time.sleep(0.3)
        
        car.throttle=0.7*turn_first
        time.sleep(0.37)
        car.throttle=0
        time.sleep(0.3)
        direction=direction*-1
        turn_first=turn_first*-1
    car.steering=0

def update(change):
    global blocked_slider, car
    
    x = change['new'] 
    x = preprocess(x)
    y = model_oa(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
   
    
    blocked_slider.value = prob_blocked
    #We need the object detection here
    if prob_blocked < 0.9:
        car.steering=0
        car.throttle=.5
        print("Going straight ahead")
    else:
        #We need the object detection here too
        car.throttle=-.8
        time.sleep(0.1)
        car.throttle=0
        print("Stopping because object detected")
        time.sleep(0.5)
        
        turn(40)

    time.sleep(0.001)
        


Cool! We've created our neural network execution function, but now we need to attach it to the camera for processing. 

We accomplish that with the ``observe`` function.

> WARNING: This code will move the robot!! Please make sure your robot has clearance.  The collision avoidance should work, but the neural
> network is only as good as the data it's trained on!

In [ ]:
update({'new': camera.value})  # we call the function once to intialize
time.sleep(.5)
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

Awesome! If your robot is plugged in it should now be generating new commands with each new camera frame.  Perhaps start by placing your robot on the ground and seeing what it does when it reaches an obstacle.

If you want to stop this behavior, you can unattach this callback by executing the code below.

In [ ]:
import time


camera.unobserve(update, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

car.throttle=0

In [ ]:
camera.unobserve(update)

Perhaps you want the robot to run without streaming video to the browser.  You can unlink the camera as below.

In [ ]:
camera_link.unlink()  # don't stream to browser (will still run camera)

To continue streaming call the following.

In [ ]:
camera_link.link()  # stream to browser (wont run camera)

### Conclusion

That's it for this live demo!  Hopefully you had some fun and your robot avoided collisions intelligently! 

If your robot wasn't avoiding collisions very well, try to spot where it fails.  The beauty is that we can collect more data for these failure scenarios
and the robot should get even better :)